### In this notebook I will try to repair hyperex amplicon extraction tool

hyperex keeps crushing when there is forward primer found upstream revese primer:

thread 'main' panicked at /home/mg/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/hyperex-0.1.1/src/utils.rs:420:45:                                                                               slice index starts at 256 but ends at 65                                                                                                                                                                 note: run with \`RUST_BACKTRACE=1` environment variable to display a backtrace                                                                                                                            Aborted (core dumped)   

I will  load the fasta seq with Biopython and then use hyperex on each read

In [151]:
from Bio import SeqIO
fasta_seq = list(SeqIO.parse("./datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/untrimmed_sequences/uchime_reference_dataset_untrimmed_16_10_2022.fasta", "fasta"))
#fasta_seq = fasta_seq[:300]
len(fasta_seq)


161335

In [169]:
import os
import subprocess
import re
from collections import Counter
from Bio import SeqIO

# Initialize variables
results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = "hyperex_temp_out.fa"
hyperex_log = "hyperex.log"
hyperex_gff = "hyperex_temp_out.gff"
hyperex_infile = "infile.fa"
# Clean up any existing temporary files
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)
if os.path.exists(hyperex_infile): os.remove(hyperex_infile)

# Command template for hyperex

# Function to count occurrences of the substring in the log file
def count_log_occurrences(log_file, substring):
    count = 0
    with open(log_file, "r") as log:
        for line in log:
            if substring in line:
                count += 1
    return count

# Process the fasta sequences
lenka = len(fasta_seq)
start_index = 0

command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out"]

cc = 0
while start_index < lenka:
    print(f"{start_index}/{lenka}, good results: {len(results)}", end="\r")
    fasta_data = "".join([f">{record.id}\n{str(record.seq)}\n" for record in fasta_seq[start_index:]])
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=fasta_data)
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]", ""), re.findall("WARN(.+)", stdout))))
    errors.append(stderr)
    #print(stderr)
    # Read results if the output file exists
    if os.path.exists(hyperex_out_fa):
        results.extend(list(SeqIO.parse(hyperex_out_fa,"fasta")))
    
    # Check for errors and adjust the start index
    if stderr:
        if os.path.exists(hyperex_log):
            log_count = count_log_occurrences(hyperex_log, "[hyperex::utils][INFO] Sequence type is DNA")
            start_index += log_count
            #print(stderr)
        if len(stderr) == 0:
            print("\nno more errors")
            break
    else:break
    if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)
    if os.path.exists(hyperex_log): os.remove(hyperex_log)
    if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
    if os.path.exists(hyperex_infile): os.remove(hyperex_infile)


# Clean up any remaining temporary files
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(hyperex_infile): os.remove(hyperex_infile)

# Print results
real_results = [i for i in results if len(i) > 0]
print("number of real results:", len(real_results)," "*10)
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("WARNINGS:")
for k,v in warning_counts.items():
    print(v,"x",k)
print("\nERROR COUNT:",len(errors), sep="\n")



number of real results: 9772           
WARNINGS:
159223 x  Sequence length is less than 1500 bp. We may not be able to find some regions
120436 x  Region  not found because primer TCCTCCGCTTATTGATATGC was not found in the sequence
1295 x  Region  not found because primer GTGARTCATCGARTCTTTG was not found in the sequence
29744 x  Region  not found because primers GTGARTCATCGARTCTTTG, TCCTCCGCTTATTGATATGC was not found in the sequence

ERROR COUNT:
13


In [174]:
results[0]
len(results)
results[:10]
for i in results:
    if len(i) <1: print(i)

Number of features: 0
Seq('')
ID: Naumovozyma_baii|JX087438|SH1313431.09FU|reps|k__Fungi;p__Ascomycota;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycetaceae;g__Naumovozyma;s__Naumovozyma_baii
Name: Naumovozyma_baii|JX087438|SH1313431.09FU|reps|k__Fungi;p__Ascomycota;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycetaceae;g__Naumovozyma;s__Naumovozyma_baii
Description: Naumovozyma_baii|JX087438|SH1313431.09FU|reps|k__Fungi;p__Ascomycota;c__Saccharomycetes;o__Saccharomycetales;f__Saccharomycetaceae;g__Naumovozyma;s__Naumovozyma_baii forward=GTGARTCATCGARTCTTTG reverse=TCCTCCGCTTATTGATATGC
Number of features: 0
Seq('')
Number of features: 0
Seq('')
ID: Pezoloma_ericae|UDB012594|SH1081240.09FU|reps|k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Leotiaceae;g__Pezoloma;s__Pezoloma_ericae
Name: Pezoloma_ericae|UDB012594|SH1081240.09FU|reps|k__Fungi;p__Ascomycota;c__Leotiomycetes;o__Helotiales;f__Leotiaceae;g__Pezoloma;s__Pezoloma_ericae
Description: Pezoloma_ericae|UDB

In [131]:
# old version
import os
import subprocess
import re
from collections import Counter
import sys

results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = f"hyperex_temp_out.fa"
hyperex_log = f"hyperex.log"
hyperex_gff = f"hyperex_temp_out.gff"

# Clean up any existing temporary files
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)

# Command template for hyperex
command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out"]

lenka = len(fasta_seq)
for i, record in enumerate(fasta_seq):
    if i % 50 == 0: print(f"{i}/{lenka}", end="\r")
    if i >0: command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out","--force"] 
    # Convert the record to FASTA format string
    fasta_data = f">{record.id}\n{str(record.seq)}\n"
    
    # Run hyperex with the FASTA data passed via stdin
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=fasta_data)
    
    # Collect warnings and errors
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]", ""), re.findall("WARN(.+)", stdout))))
    errors.extend(stderr)
    
    # Read results if the output file exists
    if os.path.exists(hyperex_out_fa):
        with open(hyperex_out_fa, "r") as fa_file:
            results.append(fa_file.read())

# Clean up any remaining temporary files
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)

print()
real_results = [i for i in results if len(i) > 0]
print("number of real results:", len(real_results))
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("ERRORS:", error_counts, "WARNINGS:", warning_counts, sep="\n")


250/300
number of real results: 15
ERRORS:
Counter()
WARNINGS:
[' Sequence length is less than 1500 bp. We may not be able to find some regions', ' Region  not found because primer TCCTCCGCTTATTGATATGC was not found in the sequence', ' Overwriting hyperex_temp_out.fa and hyperex_temp_out.gff files', ' Region  not found because primer GTGARTCATCGARTCTTTG was not found in the sequence', ' Region  not found because primers GTGARTCATCGARTCTTTG, TCCTCCGCTTATTGATATGC was not found in the sequence']


In [ ]:
# old version 2
import os
import subprocess
#from time import sleep
#import sys
import re
from collections import Counter
results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = f"hyperex_temp_out.fa"
hyperex_log = f"hyperex.log"
hyperex_gff = f"hyperex_temp_out.gff"
temp_filename = f"hyperex_temp_res.fasta"
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(temp_filename): os.remove(temp_filename)  # Delete the temporary input file

with open(temp_filename, "w") as temp_fasta:
    SeqIO.write(fasta_seq[0], temp_fasta, "fasta")
command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
warnings.extend(list(map(lambda x: x.replace("\x1b[0m]",""), re.findall("WARN(.+)",result.stdout))))
errors.extend(result.stderr)
if os.path.exists(hyperex_out_fa):
    with open(hyperex_out_fa, "r") as fa_file:
        results.append(fa_file.read())
#print(result.stdout)
#print(result.stderr)
lenka = len(fasta_seq)
command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename,"--force"]
for i, record in enumerate(fasta_seq[1:]):
    #temp_filename = f"hyperex_temp_res.fasta"
    if i % 50 == 0: print(f"{i}/{lenka}",end="\r")
    with open(temp_filename, "w") as temp_fasta:
        SeqIO.write(record, temp_fasta, "fasta")
    #if i == 0:command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]",""), re.findall("WARN(.+)",result.stdout))))
    errors.extend(result.stderr)
    #if os.path.exists(hyperex_out_fa):
    with open(hyperex_out_fa, "r") as fa_file:
        results.append(fa_file.read())

if os.path.exists(hyperex_out_fa):os.remove(hyperex_out_fa)  
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(temp_filename): os.remove(temp_filename)  # Delete the temporary input file

print()
real_results = [i for i in results if len(i) >0]
print("number of real results:",len(real_results))
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("ERRORS:",list(error_counts),"WARNINGS:","\n".join(list(warning_counts)),sep="\n")

